In [ ]:
import tensorflow_datasets as tfds

# Load 200 papers from TFDS Scientific Papers Dataset
dataset = tfds.load(
    "scientific_papers",
    split="train[:200]",
    shuffle_files=False
)

# Convert to list
papers = list(dataset.as_numpy_iterator())
print(f"Loaded {len(papers)} papers")
print(f"Available keys: {papers[0].keys()}")

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:   0%|          | 0/2 [00:47<?, ? url/s]
